# dlt-april-cal

This directory contains materials to implement a camera calibration as follows:

- April Tag fiducial markers placed at known locations in the environment which
  will define the calibrated coordinate system.

**Note of caution** This experiment was performed using materials I had
available at home and done in a "quick and dirty" approach. For "real usage",
one would ideally use better quality materials, such as sheets which are glued
to rigid and flat surface. This documentation may be updated with higher quality
images in the future.

## Overview of Experimental Setup

We have a number of april tags at known locations in the environment. These are
printed on two sheets of A4 paper, sheet "Z0" and sheet "Z1". We arrange these
so that, ideally, the paper sheets are perfectly planar and are parallel to the
z=0 plane. In fact, we define sheet "Z0" to be at the plane z=0 and "Z1" to be
at a fixed distance away (here 15 cm).

![overview.jpg](overview.jpg)

## Extracting Pixel Coordinates of the April Tags

We will generate a CSV file for each view of the april tag locations. In the
first view ("z0"), we record the positions of the April Tags in the "Z0" A4
sheet. In the second view ("z1"), we record the positions in the "Z1" A4 sheet.

### Option 1: use Strand Camera

If [Strand Camera](https://strawlab.org/strand-cam) with April Tag detection
enabled, (including as part of [Braid](https://strawlab.org/braid)), the
position of April Tags may be detected automatically and saved to CSV files
using the tools in the "April Tag Detection" panel.

### Option 2: use `gst-plugin-apriltag`

After building `gst-plugin-apriltag` [according to its
instructions](https://github.com/strawlab/strand-braid/blob/main/gst-plugin-apriltag/README.md),
here are the commands I used at the command line:

```bash
gst-launch-1.0 filesrc location=z0-0.jpg ! decodebin ! videoconvert ! apriltagdetector family=standard-41h12 ! filesink location=z0-0.jpg.csv
gst-launch-1.0 filesrc location=z0-1.jpg ! decodebin ! videoconvert ! apriltagdetector family=standard-41h12 ! filesink location=z0-1.jpg.csv
gst-launch-1.0 filesrc location=z0-2.jpg ! decodebin ! videoconvert ! apriltagdetector family=standard-41h12 ! filesink location=z0-2.jpg.csv
gst-launch-1.0 filesrc location=z1-0.jpg ! decodebin ! videoconvert ! apriltagdetector family=standard-41h12 ! filesink location=z1-0.jpg.csv
gst-launch-1.0 filesrc location=z1-1.jpg ! decodebin ! videoconvert ! apriltagdetector family=standard-41h12 ! filesink location=z1-1.jpg.csv
gst-launch-1.0 filesrc location=z1-2.jpg ! decodebin ! videoconvert ! apriltagdetector family=standard-41h12 ! filesink location=z1-2.jpg.csv

cat z0-0.jpg.csv > z0.csv
tail -n +2 z0-1.jpg.csv >> z0.csv
tail -n +2 z0-2.jpg.csv >> z0.csv

cat z1-0.jpg.csv > z1.csv
tail -n +2 z1-1.jpg.csv >> z1.csv
tail -n +2 z1-2.jpg.csv >> z1.csv
```

This generates files `z0.csv` and `z1.csv` which are here in the `data/` directory.

## Running the calibration

Below we will use Python in this notebook to actually create geometric models of our cameras (i.e. to perform the calibration.)

First, we define the known 3D coordinates of our fiducial makers (April Tags). These are constructed by design to have a known, fixed location. Therefore, we "hard code" these known locations:

In [1]:
Z1_METERS = 0.15

def id2xyz(idnum):
    
    # These values are the X and Y positions of the A4 sheets as defined in the Inkscape
    # `drawing.svg` file and rendered in the `Z0.pdf` and `Z1.pdf` file. All units are meters.
    
    x0a = -0.0935
    x1a = 0.08777
    
    x0b = -0.0838
    x1b = 0.0833
    
    y0 = -0.0938
    y1 = 0.0872
    
    za = 0.0
    zb = Z1_METERS

    rownum = idnum // 45
    colnum = idnum % 45
    
    if colnum >= 15:
        # sheet z1
        colnum -= 15
        x0 = x0b
        x1 = x1b        
        z = zb
    else:
        # sheet z0
        x0 = x0a
        x1 = x1a
        z = za
    
    xrange = x1-x0
    xnum = 13
    dx = xrange/xnum

    yrange = y1-y0
    ynum = 14
    dy = yrange/ynum
    
    x = colnum * dx + x0
    y = rownum * dy + y0
    return (x, y, z)

Next, we import the Python libraries we need.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pymvg.calibration # install with `pip install pymvg`
import pymvg.multi_camera_system
import imageio

We need to know the pixel dimensions of the camera, so here we take one of the images we saved with it.

In [3]:
cam_height, cam_width, _ = imageio.imread('data/z0-0.jpg').shape

We here read the pixel coordinates of each fiducial marker as seen by the camera and extracted by the April Tags software.

In [4]:
df_low = pd.read_csv('data/z0.csv')
df_high = pd.read_csv('data/z1.csv')

df = pd.concat((df_low,df_low,df_high))

# Create "human" names for x and y center.
df['x_px'] = df['h02']
df['y_px'] = df['h12']

We have multiple frames of data for the same 3D locations of the April Tags, and these may vary a little due to pixel noise, etc. So we average the pixel coordinate for all tags together.

In [5]:
# Average all data for the same tag.
df = df.groupby(['id'], as_index=False).mean()

Here we compute the known 3D location of each tag.

In [6]:
df['x'],df['y'],df['z'] = zip(*df['id'].map(id2xyz))

Now we can run the calibration for this camera. We use the DLT algorithm to find the best linear camera model.

In [7]:
# World coords in 3D
X = np.array((df['x'].values, df['y'].values, df['z'].values)).T
# Pixel coords in 2D
x = np.array((df['x_px'].values, df['y_px'].values)).T
# Run the calibration
dlt_results = pymvg.calibration.DLT(X, x, width=cam_width, height=cam_height)

Let's print our results:

In [8]:
for key in dlt_results:
    print(key, dlt_results[key])

cam {"name": "cam",
     "width": 1080,
     "height": 720,
     "P": [[ 1236.529440113545, 33.472107763674444, 612.1598733360305, 0 ],
           [ 0, 1195.4219088509992, 376.07459949749807, 0 ],
           [ 0, 0, 1.0, 0 ]],
     "K": [[ 1236.529440113545, 33.472107763674444, 612.1598733360305 ],
           [ 0, 1195.4219088509992, 376.07459949749807 ],
           [ 0, 0, 1.0 ]],
     "D": [ 0, 0, 0, 0, 0 ],
     "R": [[ 1.0, 0, 0 ],
           [ 0, 1.0, 0 ],
           [ 0, 0, 1.0 ]],
     "Q": [[ 0.9970827706033963, -0.023609764140094636, 0.07258462373742876 ],
           [ 0.0024396801100375603, 0.9603300480207533, 0.2788552435035395 ],
           [ -0.0762889017276805, -0.27786467552696714, 0.9575861452462006 ]],
     "translation": [ 0.0024788095729054924, -0.039395393537037846, 0.44329464984721323 ]
    }
mean_reproj_error 3.24576562286359


Let's name our specific camera for which these results apply.

In [9]:
cam1 = dlt_results['cam']
cam1.name = 'cam1'

Repeat the above steps for each camera in the camera system from the point "We here read the pixel coordinates of each fiducial marker". Create a new variable (like `cam2`, `cam3` and so on) for each camera in your camera system.

In [10]:
cameras = [cam1]
cam_system = pymvg.multi_camera_system.MultiCameraSystem(cameras)
cam_system.save_to_pymvg_file("calibration_pymvg.yaml")

TODO: 

- Show how to convert pymvg YAML file to Braid/Flydra XML calibration file.
- Use gradient descent to set distortion parameters.